<a href="https://colab.research.google.com/github/huuhung032/Re-Identification/blob/main/Human-ReID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount("/content/drive")

import os
os.chdir("/content/drive/MyDrive/deep-person-reid")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!pip install --upgrade quantecon

In [9]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [10]:
import torchreid
torchreid.models.show_avai_models()

['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'resnext50_32x4d', 'resnext101_32x8d', 'resnet50_fc512', 'se_resnet50', 'se_resnet50_fc512', 'se_resnet101', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'densenet121_fc512', 'inceptionresnetv2', 'inceptionv4', 'xception', 'resnet50_ibn_a', 'resnet50_ibn_b', 'nasnsetmobile', 'mobilenetv2_x1_0', 'mobilenetv2_x1_4', 'shufflenet', 'squeezenet1_0', 'squeezenet1_0_fc512', 'squeezenet1_1', 'shufflenet_v2_x0_5', 'shufflenet_v2_x1_0', 'shufflenet_v2_x1_5', 'shufflenet_v2_x2_0', 'mudeep', 'resnet50mid', 'hacnn', 'pcb_p6', 'pcb_p4', 'mlfn', 'osnet_x1_0', 'osnet_x0_75', 'osnet_x0_5', 'osnet_x0_25', 'osnet_ibn_x1_0', 'osnet_ain_x1_0', 'osnet_ain_x0_75', 'osnet_ain_x0_5', 'osnet_ain_x0_25']


In [11]:
# Construct data manager
datamanager = torchreid.data.ImageDataManager(
    root ='reid-data',
    sources = 'market1501',
    targets = 'market1501',
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms = ['random_flip', 'random_crop']
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
=> Loading test (target) dataset


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


  **************** Summary ****************
  source            : ['market1501']
  # source datasets : 1
  # source ids      : 751
  # source images   : 12936
  # source cameras  : 6
  target            : ['market1501']
  *****************************************




In [12]:
# Construct OSnet_x1_0 model
model = torchreid.models.build_model(
    name='osnet_x1_0',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

# Initialise optimiser and learning rate scheduler
optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

Downloading...
From: https://drive.google.com/uc?id=1LaG1EJpHrxdAxKnSCJ_i0u-nbxSAeiFY
To: /root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth
100%|██████████| 10.9M/10.9M [00:00<00:00, 122MB/s]


Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [13]:
# Construct engine
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True,
)

In [14]:
#Run model training and test
engine.run(
    save_dir='log/OSNet',
    max_epoch=20,
    eval_freq=10,
    print_freq=10,
    test_only=False,
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/20][10/404]	time 0.492 (2.240)	data 0.000 (1.695)	eta 5:01:13	loss 6.5815 (6.6324)	acc 3.1250 (0.3125)	lr 0.000300
epoch: [1/20][20/404]	time 0.498 (1.821)	data 0.001 (1.297)	eta 4:04:35	loss 6.4955 (6.6093)	acc 0.0000 (0.6250)	lr 0.000300
epoch: [1/20][30/404]	time 0.520 (1.788)	data 0.001 (1.269)	eta 3:59:52	loss 6.4211 (6.6098)	acc 3.1250 (0.5208)	lr 0.000300
epoch: [1/20][40/404]	time 0.501 (1.695)	data 0.001 (1.178)	eta 3:47:04	loss 6.4343 (6.5959)	acc 6.2500 (1.0938)	lr 0.000300
epoch: [1/20][50/404]	time 0.504 (1.699)	data 0.000 (1.187)	eta 3:47:25	loss 6.2486 (6.5630)	acc 0.0000 (1.3750)	lr 0.000300
epoch: [1/20][60/404]	time 0.491 (1.639)	data 0.001 (1.129)	eta 3:39:05	loss 6.5907 (6.5581)	acc 0.0000 (1.4583)	lr 0.000300
epoch: [1/20][70/404]	time 0.493 (1.646)	data 0.000 (1.136)	eta 3:39:40	loss 6.3189 (6.5149)	acc 3.1250 (1.9196)	lr 0.000300
epoch: [1/20][80/404]	time 0.502 (1.613)	data 0.001 (1.105)	eta 3:35:02	loss 6.4219 (6.4870)	acc 0.0000 (2.1094)	lr 0.000300


In [15]:
# Construct Resnet50 model
model = torchreid.models.build_model(
    name='resnet50',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

# Initialise optimiser and learning rate scheduler
optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [16]:
# Construct engine
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True,
)

In [17]:
#Run model training and test
engine.run(
    save_dir='log/resnet50',
    max_epoch=20,
    eval_freq=10,
    print_freq=10,
    test_only=False,
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/20][10/404]	time 0.470 (0.547)	data 0.001 (0.069)	eta 1:13:32	loss 6.9614 (6.7321)	acc 0.0000 (0.3125)	lr 0.000300
epoch: [1/20][20/404]	time 0.468 (0.508)	data 0.001 (0.035)	eta 1:08:14	loss 6.9076 (6.9389)	acc 3.1250 (0.7812)	lr 0.000300
epoch: [1/20][30/404]	time 0.469 (0.495)	data 0.001 (0.024)	eta 1:06:24	loss 6.5875 (6.8932)	acc 0.0000 (0.9375)	lr 0.000300
epoch: [1/20][40/404]	time 0.467 (0.488)	data 0.001 (0.018)	eta 1:05:25	loss 6.6633 (6.8271)	acc 0.0000 (0.9375)	lr 0.000300
epoch: [1/20][50/404]	time 0.464 (0.484)	data 0.001 (0.014)	eta 1:04:48	loss 6.7919 (6.7853)	acc 0.0000 (0.8125)	lr 0.000300
epoch: [1/20][60/404]	time 0.470 (0.481)	data 0.002 (0.012)	eta 1:04:19	loss 6.8422 (6.7526)	acc 0.0000 (0.7812)	lr 0.000300
epoch: [1/20][70/404]	time 0.470 (0.479)	data 0.000 (0.011)	eta 1:03:58	loss 6.6614 (6.7307)	acc 0.0000 (0.7589)	lr 0.000300
epoch: [1/20][80/404]	time 0.465 (0.477)	data 0.001 (0.009)	eta 1:03:39	loss 6.4525 (6.7072)	acc 0.0000 (0.8594)	lr 0.000300


In [18]:
# Construct Osnet_ain_x1_0 model
model = torchreid.models.build_model(
    name='osnet_ain_x1_0',
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

# Initialise optimiser and learning rate scheduler
optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

Downloading...
From: https://drive.google.com/uc?id=1-CaioD9NaqbHK_kzSMW8VE4_3KcsRjEo
To: /root/.cache/torch/checkpoints/osnet_ain_x1_0_imagenet.pth
100%|██████████| 10.9M/10.9M [00:00<00:00, 141MB/s]


Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_ain_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [19]:
# Construct engine
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True,
)

In [20]:
#Run model training and test
engine.run(
    save_dir='log/OSNet',
    max_epoch=20,
    eval_freq=10,
    print_freq=10,
    test_only=False,
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/20][10/404]	time 0.580 (0.618)	data 0.000 (0.078)	eta 1:23:05	loss 6.6513 (6.6477)	acc 0.0000 (0.0000)	lr 0.000300
epoch: [1/20][20/404]	time 0.502 (0.581)	data 0.000 (0.039)	eta 1:18:06	loss 6.7285 (6.6367)	acc 0.0000 (0.6250)	lr 0.000300
epoch: [1/20][30/404]	time 0.500 (0.556)	data 0.001 (0.026)	eta 1:14:32	loss 6.5120 (6.6186)	acc 0.0000 (0.8333)	lr 0.000300
epoch: [1/20][40/404]	time 0.498 (0.542)	data 0.001 (0.020)	eta 1:12:38	loss 6.4520 (6.5917)	acc 3.1250 (1.1719)	lr 0.000300
epoch: [1/20][50/404]	time 0.503 (0.534)	data 0.001 (0.016)	eta 1:11:29	loss 6.5192 (6.5916)	acc 0.0000 (1.2500)	lr 0.000300
epoch: [1/20][60/404]	time 0.504 (0.529)	data 0.000 (0.014)	eta 1:10:42	loss 6.2755 (6.5625)	acc 0.0000 (1.3021)	lr 0.000300
epoch: [1/20][70/404]	time 0.500 (0.525)	data 0.001 (0.012)	eta 1:10:06	loss 6.3127 (6.5343)	acc 9.3750 (1.7411)	lr 0.000300
epoch: [1/20][80/404]	time 0.504 (0.522)	data 0.000 (0.010)	eta 1:09:37	loss 5.9832 (6.4770)	acc 6.2500 (2.4219)	lr 0.000300
